<a href="https://colab.research.google.com/github/half-anerd/spam_detection/blob/main/spam_svm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
data = pd.read_csv('/content/spam.csv', encoding='ISO-8859-1')
data[:5]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [ ]:
data.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'],axis='columns',inplace=True)
data.columns = ['label', 'message']
data.describe()

,label,message
count,5572,5572
unique,2,5169
top,ham,"Sorry, I'll call later"
freq,4825,30


In [ ]:
ps = PorterStemmer()
corpus = []
for i in range(0, len(data)):
    review = re.sub('[^a-zA-Z]', ' ', data['message'][i])
    review = review.lower()
    review = nltk.word_tokenize(review)  
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)
corpus[:5]    

['go jurong point crazi avail bugi n great world la e buffet cine got amor wat',
 'ok lar joke wif u oni',
 'free entri wkli comp win fa cup final tkt st may text fa receiv entri question std txt rate c appli',
 'u dun say earli hor u c alreadi say',
 'nah think goe usf live around though']

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 5000)
x = cv.fit_transform(corpus).toarray()
y = pd.get_dummies(data['label'])
y = y.iloc[:,1].values

print(data['label'].head(),"\n y = ",y.shape)
print("x = ", x.shape)

0     ham
1     ham
2    spam
3     ham
4     ham
Name: label, dtype: object 
 y =  (5572,)
x =  (5572, 5000)


In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2, random_state=0)

In [ ]:
from sklearn import svm
clf = svm.SVC(kernel='rbf', C=1.0)
clf.fit(x_train, y_train)
predictions = clf.predict(x_test)

In [ ]:
error = np.array(y_test.shape, dtype="int8")
error = predictions - y_test
false_positives=0 
false_negatives=0
for i in range(len(error)):
  if (error[i]==1):
    false_positives+=1
  elif (error[i]==255):
    false_negatives+=1
    error[i]/=255
false_positives, false_negatives      

(0, 26)

In [ ]:
accuracy = 1 - sum(error)/len(y_test)
accuracy

0.9766816143497757